In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm


In [10]:
df  = pd.read_csv("data/data.csv")
df.head(3)

,filename,imagefilepath,targetfilepath,imagesize,tablecount,tables,bndboxlim
0,00001.PNG,data/images/00001.PNG,data/labels/00001.xml,['1126' '1596' '3'],1,"[OrderedDict([('name', 'Table'), ('pose', 'Uns...",[['15' '442' '1095' '1467' '1']]
1,00005.PNG,data/images/00005.PNG,data/labels/00005.xml,['1063' '844' '3'],1,"[OrderedDict([('name', 'Table'), ('pose', 'Uns...",[['327' '543' '904' '834' '1']]
2,00006.PNG,data/images/00006.PNG,data/labels/00006.xml,['682' '845' '3'],5,"[OrderedDict([('name', 'Table'), ('pose', 'Uns...",[['18' '56' '267' '256' '1']\n ['331' '111' '5...


In [11]:
df['bndboxlim'][0]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   filename        340 non-null    object
 1   imagefilepath   340 non-null    object
 2   targetfilepath  340 non-null    object
 3   imagesize       340 non-null    object
 4   tablecount      340 non-null    int64 
 5   tables          340 non-null    object
 6   bndboxlim       340 non-null    object
dtypes: int64(1), object(6)
memory usage: 18.7+ KB
